In [1]:
#import
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import numpy as np
import pymysql
import operator
import os
from scipy import stats
from statsmodels.regression.linear_model import OLS
from datetime import *
import scipy.io as sio
import h5py
import copy
import shutil
import rarfile
from general_function import *
import scipy.stats as scs
from matplotlib_venn import venn3
from matplotlib_venn import venn2
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.font_manager import FontProperties
plt.rcParams['font.sans-serif'] = ['SimHei']  #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  #用来正常显示负号
sns.set(style="whitegrid")
myfont = FontProperties(fname=r'C:\Windows\Fonts\simhei.ttf', size=10)
sns.set(
    font=myfont.get_name(),
    style="whitegrid",
    palette="muted",
    color_codes=True)

In [2]:
#data and preprocess 1
zzIcData = pd.read_csv('zz500_from_db.csv')
zzIcData.columns = [int(i) for i in zzIcData.columns]
zzIcData[1] = [pd.datetime.strptime(str(date.fromordinal(int(i)-366)).replace('-',''),'%Y%m%d') for i in zzIcData[1]]
zzIcData = zzIcData[zzIcData[1]>=datetime(2015,1,1)]
zzIcData.set_index(1,inplace=True)
benchmark_return = (zzIcData[5]/zzIcData[5].shift(1)-1).dropna()

In [3]:
#data and preprocess 2
dateparse = lambda dates: pd.datetime.strptime(dates, '%Y%m%d')
mkt_cap_ard = pd.read_csv('mkt_cap_ard.csv',index_col=0,date_parser=dateparse).replace(0,np.nan)
mkt_cap_ard.columns = mkt_cap_ard.columns.astype(int)
mkt_cap_ard.fillna(method='ffill',inplace=True)
mkt_cap_ard = mkt_cap_ard[mkt_cap_ard.index>=datetime(2015,1,1)]
volume = pd.read_csv('volume.csv',index_col=0,date_parser=dateparse)
volume.columns = volume.columns.astype(int)
volume.fillna(0,inplace=True)
free_float_shares = pd.read_csv('free_float_shares.csv',index_col=0,date_parser=dateparse)
free_float_shares.columns = free_float_shares.columns.astype(int)
free_float_shares.fillna(method='ffill',inplace=True)
free_float_turn = volume/free_float_shares

In [4]:
#data and preprocess 3
pcf_ocf_ttm = pd.read_csv('pcf_ocf_ttm.csv',index_col=0,date_parser=dateparse)
pcf_ocf_ttm.columns = pcf_ocf_ttm.columns.astype(int)
pcf_ocf_ttm = pcf_ocf_ttm[pcf_ocf_ttm.index>=datetime(2015,1,1)]
pb_lf = pd.read_csv('pb_lf.csv',index_col=0,date_parser=dateparse)
pb_lf.columns = pb_lf.columns.astype(int)
pb_lf = pb_lf[pb_lf.index>=datetime(2015,1,1)]
pe_ttm = pd.read_csv('pe_ttm.csv',index_col=0,date_parser=dateparse)
pe_ttm.columns = pe_ttm.columns.astype(int)
pe_ttm = pe_ttm[pe_ttm.index>=datetime(2015,1,1)]
qfa_yoyprofit = pd.read_csv('qfa_yoyprofit.csv',index_col=0,date_parser=dateparse)
qfa_yoyprofit.columns = qfa_yoyprofit.columns.astype(int)
qfa_yoyprofit = qfa_yoyprofit[qfa_yoyprofit.index>=datetime(2015,1,1)]
qfa_yoysales = pd.read_csv('qfa_yoysales.csv',index_col=0,date_parser=dateparse)
qfa_yoysales.columns = qfa_yoysales.columns.astype(int)
qfa_yoysales = qfa_yoysales[qfa_yoysales.index>=datetime(2015,1,1)]
tot_liab = pd.read_csv('tot_liab.csv',index_col=0,date_parser=dateparse)
tot_liab.columns = tot_liab.columns.astype(int)
tot_liab = tot_liab[tot_liab.index>=datetime(2015,1,1)]
tot_assets = pd.read_csv('tot_assets.csv',index_col=0,date_parser=dateparse)
tot_assets.columns = tot_assets.columns.astype(int)
tot_assets = tot_assets[tot_assets.index>=datetime(2015,1,1)]
tot_non_cur_liab = pd.read_csv('tot_non_cur_liab.csv',index_col=0,date_parser=dateparse)
tot_non_cur_liab.columns = tot_non_cur_liab.columns.astype(int)
tot_non_cur_liab = tot_non_cur_liab[tot_non_cur_liab.index>=datetime(2015,1,1)]
tot_equity = pd.read_csv('tot_equity.csv',index_col=0,date_parser=dateparse)
tot_equity.columns = tot_equity.columns.astype(int)
tot_equity = tot_equity[tot_equity.index>=datetime(2015,1,1)]

In [5]:
#data and preprocess 4
close_p = pd.read_csv('close.csv',index_col = 0, date_parser=dateparse)
amt = pd.read_csv('amt.csv',index_col = 0, date_parser=dateparse)
adjfactor = pd.read_csv('adjfactor.csv',index_col = 0, date_parser=dateparse)
close_p.columns = close_p.columns.astype(int)
amt.columns = amt.columns.astype(int)
adjfactor.columns = adjfactor.columns.astype(int)
close_p = close_p[close_p.index>=datetime(2015,1,1)]
amt = amt[amt.index>=datetime(2015,1,1)]
adjfactor = adjfactor[adjfactor.index>=datetime(2015,1,1)]
adjclose = adjfactor * close_p
adjclose.fillna(method='ffill',inplace=True)
adjpreclose = adjclose.shift(1)
close_to_close = adjclose/adjpreclose - 1
amt = amt.replace(0,np.nan)
amt = amt.T.dropna(how='all').T
#close_to_close = close_to_close[pd.notnull(amt)].dropna(how='all')
close_to_close = close_to_close.dropna(how='all')

In [6]:
#data and preprocess 5
def expontial_decay_weight(n,half_life,normalize = False):
    alpha = 1 - np.exp((np.log(0.5))/half_life)
    weight_array = np.arange(n-1,-1,-1)
    weight_array = (1-alpha)**weight_array
    if normalize:
        weight_array = weight_array/weight_array.sum()
    return weight_array
benchmark_return_df = pd.DataFrame()
for i in close_to_close.columns:
    add = benchmark_return.copy()
    add.name = i
    benchmark_return_df = pd.concat([benchmark_return_df,add],axis=1)
excess_close_to_close = close_to_close - benchmark_return_df

In [7]:
def Schimidt(factors_ori):
    col_name = factors_ori.columns
    factors = factors_ori.values

    R = np.zeros((factors.shape[1], factors.shape[1]))
    Q = np.zeros(factors.shape)
    for k in range(0, factors.shape[1]):
        R[k, k] = np.sqrt(np.dot(factors[:, k], factors[:, k]))
        Q[:, k] = factors[:, k]/R[k, k]
        for j in range(k+1, factors.shape[1]):
            R[k, j] = np.dot(Q[:, k], factors[:, j])
            factors[:, j] = factors[:, j] - R[k, j]*Q[:, k]

    Q = pd.DataFrame(Q,columns = col_name,index = factors_ori.index)
    return Q

In [8]:
SIZE = np.log(mkt_cap_ard)

In [9]:
def process(df):
    df = df[:-21]
    df[np.isnan(df)]=0
    weight = expontial_decay_weight(len(df),126,normalize=True)
    return np.sum(np.log(df+1)*weight)
RSTR = close_to_close.rolling(window = 504, min_periods=42).apply(process)

In [10]:
def process(df):
    df[np.isnan(df)]=0
    weight = expontial_decay_weight(len(df),42,normalize=True)
    return np.sqrt(np.sum(np.square(df)*weight))
DASTD = excess_close_to_close.rolling(window = 252, min_periods=42).apply(process)

In [11]:
def process(df):
    df[np.isnan(df)]=0
    Z_12 = np.exp(np.sum(np.log(1+df)))-1
    Z_11 = np.exp(np.sum(np.log(1+df[1*21:])))-1
    Z_10 = np.exp(np.sum(np.log(1+df[2*21:])))-1
    Z_9 = np.exp(np.sum(np.log(1+df[3*21:])))-1
    Z_8 = np.exp(np.sum(np.log(1+df[4*21:])))-1
    Z_7 = np.exp(np.sum(np.log(1+df[5*21:])))-1
    Z_6 = np.exp(np.sum(np.log(1+df[6*21:])))-1
    Z_5 = np.exp(np.sum(np.log(1+df[7*21:])))-1
    Z_4 = np.exp(np.sum(np.log(1+df[8*21:])))-1
    Z_3 = np.exp(np.sum(np.log(1+df[9*21:])))-1
    Z_2 = np.exp(np.sum(np.log(1+df[10*21:])))-1
    Z_1 = np.exp(np.sum(np.log(1+df[11*21:])))-1
    return np.log(1 + np.max([Z_12,Z_11,Z_10,Z_9,Z_8,Z_7,Z_6,Z_5,Z_4,Z_3,Z_2,Z_1])) - \
           np.log(1 + np.min([Z_12,Z_11,Z_10,Z_9,Z_8,Z_7,Z_6,Z_5,Z_4,Z_3,Z_2,Z_1]))
CMRA = close_to_close.rolling(window = 12*21).apply(process)

In [12]:
beta_calculation_mid = pd.concat([close_to_close.stack(),benchmark_return_df.stack()],axis=1)
cov_df = pd.DataFrame()
for i in close_to_close.columns.tolist():
    add = beta_calculation_mid.xs(i,level=1).ewm(min_periods = 42, halflife=63).cov().xs(1,level=1)[0]
    add.name = i
    cov_df = pd.concat([cov_df,add],axis=1)
BETA = cov_df/(benchmark_return_df.ewm(min_periods = 42, halflife=63).std()**2)

In [13]:
HSIGMA = pd.DataFrame()
for i in close_to_close.index:
    sub_close_to_close = close_to_close[close_to_close.index<=i]
    sub_benchmark_return_df =benchmark_return_df[benchmark_return_df.index<=i]
    add = (sub_close_to_close.iloc[-252:] - sub_benchmark_return_df.iloc[-252:] * BETA.loc[i]).std()
    add.name = i
    HSIGMA = pd.concat([HSIGMA,add.to_frame().T])

In [14]:
NonLinerSize = (1/np.square((SIZE.T - SIZE.mean(axis=1))/SIZE.std(axis=1))).T

In [15]:
STOM = np.log(free_float_turn.replace(0,np.nan).rolling(window=21).mean())
STOQ = np.log(free_float_turn.replace(0,np.nan).rolling(window=63).mean())
STOA = np.log(free_float_turn.replace(0,np.nan).rolling(window=252).mean())

In [16]:
BP = 1/pb_lf
CETOP = 1/pcf_ocf_ttm
ETOP = 1/pe_ttm
YOYProfit = qfa_yoyprofit
YOYSales = qfa_yoysales
MLEV = (mkt_cap_ard + tot_non_cur_liab) / mkt_cap_ard
DTOA = tot_assets / tot_liab
BLEV = (tot_equity + tot_non_cur_liab) / tot_equity

In [17]:
#origin
AGG_BETA = 1 * BETA
AGG_SIZE = 1 * SIZE
AGG_MOMENTUM = 1 * RSTR
AGG_VOLATILITY = (0.7 * DASTD.fillna(0) + 0.15 * CMRA.fillna(0) + 0.15 * HSIGMA.fillna(0)).replace(0, np.nan)
AGG_NonLinerSize = 1 * NonLinerSize
AGG_VALUE = 1 * BP
AGG_LIQUIDITY = (0.5 * STOM.fillna(0) + 0.25 * STOQ.fillna(0) + 0.25 * STOA.fillna(0)).replace(0, np.nan)
AGG_PROFITABILITY = (0.5 * CETOP.fillna(0) + 0.5 * ETOP.fillna(0)).replace(0, np.nan)
AGG_GROWTH = (0.5 * YOYProfit.fillna(0) + 0.5 * YOYSales.fillna(0)).replace(0, np.nan)
AGG_LEVERAGE = (1/3 * MLEV.fillna(0) + 1/3 * DTOA.fillna(0) + 1/3 * BLEV.fillna(0)).replace(0, np.nan)

In [18]:
#write
AGG_BETA.dropna(how='all').to_csv('Non_Neutralized_BETA.csv',encoding='gbk')
AGG_SIZE.dropna(how='all').to_csv('Non_Neutralized_SIZE.csv',encoding='gbk')
AGG_MOMENTUM.dropna(how='all').to_csv('Non_Neutralized_MOMENTUM.csv',encoding='gbk')
AGG_VOLATILITY.dropna(how='all').to_csv('Non_Neutralized_VOLATILITY.csv',encoding='gbk')
AGG_NonLinerSize.dropna(how='all').to_csv('Non_Neutralized_NonLinerSize.csv',encoding='gbk')
AGG_VALUE.dropna(how='all').to_csv('Non_Neutralized_VALUE.csv',encoding='gbk')
AGG_LIQUIDITY.dropna(how='all').to_csv('Non_Neutralized_LIQUIDITY.csv',encoding='gbk')
AGG_PROFITABILITY.dropna(how='all').to_csv('Non_Neutralized_PROFITABILITY.csv',encoding='gbk')
AGG_GROWTH.dropna(how='all').to_csv('Non_Neutralized_GROWTH.csv',encoding='gbk')
AGG_LEVERAGE.dropna(how='all').to_csv('Non_Neutralized_LEVERAGE.csv',encoding='gbk')

In [19]:
#neutralize
AGG_BETA = ((AGG_BETA.T - (((mkt_cap_ard[pd.notnull(AGG_BETA)].T/\
    mkt_cap_ard[pd.notnull(AGG_BETA)].sum(axis=1)).T)* AGG_BETA).sum(axis=1))/AGG_BETA.std(axis=1)).T
AGG_SIZE = ((AGG_SIZE.T - (((mkt_cap_ard[pd.notnull(AGG_SIZE)].T/\
    mkt_cap_ard[pd.notnull(AGG_SIZE)].sum(axis=1)).T)* AGG_SIZE).sum(axis=1))/AGG_SIZE.std(axis=1)).T
AGG_MOMENTUM = ((AGG_MOMENTUM.T - (((mkt_cap_ard[pd.notnull(AGG_MOMENTUM)].T/\
    mkt_cap_ard[pd.notnull(AGG_MOMENTUM)].sum(axis=1)).T)* AGG_MOMENTUM).sum(axis=1))/AGG_MOMENTUM.std(axis=1)).T
AGG_VOLATILITY = ((AGG_VOLATILITY.T - (((mkt_cap_ard[pd.notnull(AGG_VOLATILITY)].T/\
    mkt_cap_ard[pd.notnull(AGG_VOLATILITY)].sum(axis=1)).T)* AGG_VOLATILITY).sum(axis=1))/AGG_VOLATILITY.std(axis=1)).T
AGG_NonLinerSize = ((AGG_NonLinerSize.T - (((mkt_cap_ard[pd.notnull(AGG_NonLinerSize)].T/\
    mkt_cap_ard[pd.notnull(AGG_NonLinerSize)].sum(axis=1)).T)* AGG_NonLinerSize).sum(axis=1))/AGG_NonLinerSize.std(axis=1)).T
AGG_VALUE = ((AGG_VALUE.T - (((mkt_cap_ard[pd.notnull(AGG_VALUE)].T/\
    mkt_cap_ard[pd.notnull(AGG_VALUE)].sum(axis=1)).T)* AGG_VALUE).sum(axis=1))/AGG_VALUE.std(axis=1)).T
AGG_LIQUIDITY = ((AGG_LIQUIDITY.T - (((mkt_cap_ard[pd.notnull(AGG_LIQUIDITY)].T/\
    mkt_cap_ard[pd.notnull(AGG_LIQUIDITY)].sum(axis=1)).T)* AGG_LIQUIDITY).sum(axis=1))/AGG_LIQUIDITY.std(axis=1)).T
AGG_PROFITABILITY = ((AGG_PROFITABILITY.T - (((mkt_cap_ard[pd.notnull(AGG_PROFITABILITY)].T/\
    mkt_cap_ard[pd.notnull(AGG_PROFITABILITY)].sum(axis=1)).T)* AGG_PROFITABILITY).sum(axis=1))/AGG_PROFITABILITY.std(axis=1)).T
AGG_GROWTH = ((AGG_GROWTH.T - (((mkt_cap_ard[pd.notnull(AGG_GROWTH)].T/\
    mkt_cap_ard[pd.notnull(AGG_GROWTH)].sum(axis=1)).T)* AGG_GROWTH).sum(axis=1))/AGG_GROWTH.std(axis=1)).T
AGG_LEVERAGE = ((AGG_LEVERAGE.T - (((mkt_cap_ard[pd.notnull(AGG_LEVERAGE)].T/\
    mkt_cap_ard[pd.notnull(AGG_LEVERAGE)].sum(axis=1)).T)* AGG_LEVERAGE).sum(axis=1))/AGG_LEVERAGE.std(axis=1)).T

In [20]:
#write
AGG_BETA.dropna(how='all').to_csv('AGG_BETA.csv',encoding='gbk')
AGG_SIZE.dropna(how='all').to_csv('AGG_SIZE.csv',encoding='gbk')
AGG_MOMENTUM.dropna(how='all').to_csv('AGG_MOMENTUM.csv',encoding='gbk')
AGG_VOLATILITY.dropna(how='all').to_csv('AGG_VOLATILITY.csv',encoding='gbk')
AGG_NonLinerSize.dropna(how='all').to_csv('AGG_NonLinerSize.csv',encoding='gbk')
AGG_VALUE.dropna(how='all').to_csv('AGG_VALUE.csv',encoding='gbk')
AGG_LIQUIDITY.dropna(how='all').to_csv('AGG_LIQUIDITY.csv',encoding='gbk')
AGG_PROFITABILITY.dropna(how='all').to_csv('AGG_PROFITABILITY.csv',encoding='gbk')
AGG_GROWTH.dropna(how='all').to_csv('AGG_GROWTH.csv',encoding='gbk')
AGG_LEVERAGE.dropna(how='all').to_csv('AGG_LEVERAGE.csv',encoding='gbk')